In [1]:
import os
import pandas as pd

# Define paths for data
base_folder = "C:/FRR40/rat25-main/temp"
detections_folder = os.path.join(base_folder, "detections")
ego_motion_folder = os.path.join(base_folder, "ego_motion")
objects_folder = os.path.join(base_folder, "objects")

# Load files into dictionaries
def load_files_into_dict(folder_path):
    files = sorted([f for f in os.listdir(folder_path) if f.endswith(".p")])
    data_dict = {file: pd.read_pickle(os.path.join(folder_path, file)) for file in files}
    return data_dict

# Load data
detections_data = load_files_into_dict(detections_folder)
ego_motion_data = load_files_into_dict(ego_motion_folder)
objects_data = load_files_into_dict(objects_folder)

# Inspect a sample file from each type
print("Sample Detection File:")
print(detections_data[next(iter(detections_data))].head())
print("\nSample Ego Motion File:")
print(ego_motion_data[next(iter(ego_motion_data))].head())
print("\nSample Object File:")
print(objects_data[next(iter(objects_data))].head())


Sample Detection File:
                   cRC  length  counter    dataID  radialVelocityDomainMin  \
0  3320392428831458305   33715     4919  16777389               -25.593336   
1   583909460173523723   33715     4920  16777389               -23.288147   
2  -235517240581128623   33715     4921  16777389               -28.015123   
3  9029336534572036161   33715     4922  16777389               -24.840691   
4  2992310922064066122   33715     4923  16777389               -23.058084   

   numberOfDetections  azimuthCorrection  radialVelocityDomainMax  \
0                 202          -0.011373                25.593336   
1                 211          -0.011373                23.288147   
2                 213          -0.011373                28.015123   
3                 209          -0.011373                24.840691   
4                 219          -0.011373                23.058084   

   misalignmentProbability  elevationCorrection  ...  \
0                        0           

In [2]:
# Define relevant columns for each file type (updated for rcs in detections)
columns_to_keep = {
    "detections": [
        "timestamp", "rcs", "distance", "angleAzimuth", "angleElevation", 
        "radialVelocity", "radialVelocityDomainMax"
    ],
    "ego_motion": [
        "timestamp", "RotationRates.yawRateVehicleBody.value", "Velocity.SpeedCog.SpeedCog"
    ],
    "objects": [
        "timestamp", "orientation", "x", "y", "width_edge_mean", "length_edge_mean",
        "status_measurement", "status_movement", "overdrivable", "underdrivable",
        "header.origin.x", "header.origin.y", "header.origin.yaw"
    ]
}

# Function to filter relevant columns
def filter_columns(data_dict, columns_to_keep):
    filtered_data = {}
    for file, df in data_dict.items():
        print(f"Filtering columns for file: {file}")
        filtered_data[file] = df[columns_to_keep]
    return filtered_data

# Apply filtering
filtered_detections = filter_columns(detections_data, columns_to_keep["detections"])
filtered_ego_motion = filter_columns(ego_motion_data, columns_to_keep["ego_motion"])
filtered_objects = filter_columns(objects_data, columns_to_keep["objects"])

# Save filtered data for later use
filtered_folder = "C:/FRR40/rat25-main/filtered"
os.makedirs(filtered_folder, exist_ok=True)

def save_filtered_data(filtered_data, folder_name):
    save_path = os.path.join(filtered_folder, folder_name)
    os.makedirs(save_path, exist_ok=True)
    for file, df in filtered_data.items():
        df.to_pickle(os.path.join(save_path, file))
    print(f"Filtered files saved in: {save_path}")

save_filtered_data(filtered_detections, "detections")
save_filtered_data(filtered_ego_motion, "ego_motion")
save_filtered_data(filtered_objects, "objects")


Filtering columns for file: frr40_detections_0.p
Filtering columns for file: frr40_detections_1.p
Filtering columns for file: frr40_detections_10.p
Filtering columns for file: frr40_detections_11.p
Filtering columns for file: frr40_detections_12.p
Filtering columns for file: frr40_detections_13.p
Filtering columns for file: frr40_detections_14.p
Filtering columns for file: frr40_detections_15.p
Filtering columns for file: frr40_detections_2.p
Filtering columns for file: frr40_detections_3.p
Filtering columns for file: frr40_detections_4.p
Filtering columns for file: frr40_detections_5.p
Filtering columns for file: frr40_detections_6.p
Filtering columns for file: frr40_detections_7.p
Filtering columns for file: frr40_detections_8.p
Filtering columns for file: frr40_detections_9.p
Filtering columns for file: egoMotionDynamicData_0.p
Filtering columns for file: egoMotionDynamicData_1.p
Filtering columns for file: egoMotionDynamicData_10.p
Filtering columns for file: egoMotionDynamicData_1

In [3]:
import numpy as np

def transform_to_scs(row):
    """
    Transform VCS coordinates to SCS coordinates.
    Args:
        row: A single row of the DataFrame containing VCS coordinates and offsets.
    Returns:
        x_scs, y_scs: Transformed coordinates in SCS.
    """
    # Extract VCS coordinates
    x_vcs = np.array(row["x"])
    y_vcs = np.array(row["y"])

    # Extract origin offsets and yaw
    x_offset = row.get("header.origin.x", 0)
    y_offset = row.get("header.origin.y", 0)
    yaw = row.get("header.origin.yaw", 0)

    # Translate coordinates to the sensor origin
    x_translated = x_vcs - x_offset
    y_translated = y_vcs - y_offset

    # Apply rotation (yaw)
    cos_yaw = np.cos(yaw)
    sin_yaw = np.sin(yaw)
    x_scs = cos_yaw * x_translated + sin_yaw * y_translated
    y_scs = -sin_yaw * x_translated + cos_yaw * y_translated

    return x_scs, y_scs

def apply_vcs_to_scs(objects_data):
    """
    Apply VCS to SCS transformation to all objects files.
    Args:
        objects_data: Dictionary of filtered objects DataFrames.
    Returns:
        transformed_objects: Dictionary with transformed SCS coordinates.
    """
    transformed_objects = {}
    for file, df in objects_data.items():
        print(f"Transforming VCS to SCS for file: {file}")
        df = df.copy()
        df[["x_scs", "y_scs"]] = df.apply(
            lambda row: pd.Series(transform_to_scs(row)), axis=1
        )
        # Drop original VCS columns as they are no longer needed
        df.drop(columns=["x", "y", "header.origin.x", "header.origin.y", "header.origin.yaw"], inplace=True)
        transformed_objects[file] = df
    return transformed_objects

# Apply VCS to SCS transformation
transformed_objects = apply_vcs_to_scs(filtered_objects)

# Save the transformed objects
transformed_objects_folder = "C:/FRR40/rat25-main/transformed_objects"
os.makedirs(transformed_objects_folder, exist_ok=True)

for file, df in transformed_objects.items():
    df.to_pickle(os.path.join(transformed_objects_folder, file))

print(f"\nTransformed objects saved in: {transformed_objects_folder}")


Transforming VCS to SCS for file: frr40_objects_0.p
Transforming VCS to SCS for file: frr40_objects_1.p
Transforming VCS to SCS for file: frr40_objects_10.p
Transforming VCS to SCS for file: frr40_objects_11.p
Transforming VCS to SCS for file: frr40_objects_12.p
Transforming VCS to SCS for file: frr40_objects_13.p
Transforming VCS to SCS for file: frr40_objects_14.p
Transforming VCS to SCS for file: frr40_objects_15.p
Transforming VCS to SCS for file: frr40_objects_2.p
Transforming VCS to SCS for file: frr40_objects_3.p
Transforming VCS to SCS for file: frr40_objects_4.p
Transforming VCS to SCS for file: frr40_objects_5.p
Transforming VCS to SCS for file: frr40_objects_6.p
Transforming VCS to SCS for file: frr40_objects_7.p
Transforming VCS to SCS for file: frr40_objects_8.p
Transforming VCS to SCS for file: frr40_objects_9.p

Transformed objects saved in: C:/FRR40/rat25-main/transformed_objects


In [10]:
import matplotlib.pyplot as plt

def plot_vcs_vs_scs_coordinates(data, file_name=None):
    """
    Plot the original VCS (x, y) and transformed SCS (x_scs, y_scs) coordinates.
    Args:
        data: Dictionary of DataFrames containing VCS and SCS coordinates.
        file_name: Optional, specify the file to plot from the data dictionary.
    """
    if file_name is None:
        # Choose a random file if not specified
        file_name = list(data.keys())[0]

    df = data[file_name]
    print(f"Visualizing VCS vs. SCS coordinates for file: {file_name}")

    # Iterate through each row (representing a radar cycle)
    for index, row in df.iterrows():
        if "x" in row and "y" in row and "x_scs" in row and "y_scs" in row:
            # Extract coordinates
            x_vcs = row["x"]
            y_vcs = row["y"]
            x_scs = row["x_scs"]
            y_scs = row["y_scs"]

            # Check if data is valid
            if not (isinstance(x_vcs, list) and isinstance(y_vcs, list) and 
                    isinstance(x_scs, list) and isinstance(y_scs, list)):
                print(f"Skipping cycle {index} due to invalid data format.")
                continue

            # Create the plot
            plt.figure(figsize=(12, 8))
            
            # Plot VCS coordinates
            plt.scatter(x_vcs, y_vcs, c="blue", alpha=0.6, label="Original (VCS)")
            
            # Plot SCS coordinates
            plt.scatter(x_scs, y_scs, c="red", alpha=0.6, label="Transformed (SCS)")
            
            # Plot settings
            plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
            plt.axvline(0, color='black', linewidth=0.8, linestyle='--')
            plt.title(f"VCS vs. SCS Coordinates - File: {file_name}, Cycle: {index}")
            plt.xlabel("X Coordinate (m)")
            plt.ylabel("Y Coordinate (m)")
            plt.legend()
            plt.grid(True)
            plt.show()

            # Plot only the first valid cycle for simplicity
            break

# Example usage
plot_vcs_vs_scs_coordinates(transformed_objects)


Visualizing VCS vs. SCS coordinates for file: frr40_objects_0.p


In [11]:
def plot_vcs_vs_scs_coordinates_debug(data, file_name=None):
    """
    Plot the original VCS (x, y) and transformed SCS (x_scs, y_scs) coordinates.
    Adds debugging information to identify issues with the data.
    Args:
        data: Dictionary of DataFrames containing VCS and SCS coordinates.
        file_name: Optional, specify the file to plot from the data dictionary.
    """
    if file_name is None:
        # Choose a random file if not specified
        file_name = list(data.keys())[0]

    df = data[file_name]
    print(f"Visualizing VCS vs. SCS coordinates for file: {file_name}")

    # Debugging: Check the structure of the DataFrame
    print("DataFrame Columns:", df.columns)
    print("Sample DataFrame Head:")
    print(df.head())

    # Iterate through each row (representing a radar cycle)
    for index, row in df.iterrows():
        # Check if required columns exist
        if all(col in row for col in ["x", "y", "x_scs", "y_scs"]):
            # Extract coordinates
            x_vcs = row["x"]
            y_vcs = row["y"]
            x_scs = row["x_scs"]
            y_scs = row["y_scs"]

            # Debugging: Check the validity of the data
            if not (isinstance(x_vcs, list) and isinstance(y_vcs, list) and 
                    isinstance(x_scs, list) and isinstance(y_scs, list)):
                print(f"Skipping cycle {index}: Invalid data format.")
                print(f"x_vcs type: {type(x_vcs)}, y_vcs type: {type(y_vcs)}, x_scs type: {type(x_scs)}, y_scs type: {type(y_scs)}")
                continue

            if len(x_vcs) != len(y_vcs) or len(x_scs) != len(y_scs):
                print(f"Skipping cycle {index}: Mismatched lengths.")
                print(f"x_vcs length: {len(x_vcs)}, y_vcs length: {len(y_vcs)}, x_scs length: {len(x_scs)}, y_scs length: {len(y_scs)}")
                continue

            # Create the plot
            plt.figure(figsize=(12, 8))
            
            # Plot VCS coordinates
            plt.scatter(x_vcs, y_vcs, c="blue", alpha=0.6, label="Original (VCS)")
            
            # Plot SCS coordinates
            plt.scatter(x_scs, y_scs, c="red", alpha=0.6, label="Transformed (SCS)")
            
            # Plot settings
            plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
            plt.axvline(0, color='black', linewidth=0.8, linestyle='--')
            plt.title(f"VCS vs. SCS Coordinates - File: {file_name}, Cycle: {index}")
            plt.xlabel("X Coordinate (m)")
            plt.ylabel("Y Coordinate (m)")
            plt.legend()
            plt.grid(True)
            plt.show()

            # Plot only the first valid cycle for simplicity
            break
        else:
            print(f"Cycle {index}: Missing required columns.")

# Example usage with debugging
plot_vcs_vs_scs_coordinates_debug(transformed_objects)


Visualizing VCS vs. SCS coordinates for file: frr40_objects_0.p
DataFrame Columns: Index(['timestamp', 'orientation', 'width_edge_mean', 'length_edge_mean',
       'status_measurement', 'status_movement', 'overdrivable',
       'underdrivable', 'x_scs', 'y_scs'],
      dtype='object')
Sample DataFrame Head:
      timestamp                                        orientation  \
0  2.739721e+07  [3.0897224, -1.5708438, -1.5708438, 0.0, 0.0, ...   
1  2.739721e+07  [3.141304, -0.8097746, -1.5708438, 0.0, 0.0, 0...   
2  2.739721e+07  [-0.042569254, -0.8095828, -0.99836403, 0.0, 0...   
3  2.739721e+07  [-0.048321854, -0.9982682, 2.328078, 0.0, 0.0,...   
4  2.739721e+07  [3.0900102, 2.2941377, 0.0, 0.0, 0.0, 0.0, 0.0...   

                                     width_edge_mean  \
0  [0.17999999, 2.01, 0.66999996, 0.71, 1.74, 0.2...   
1  [0.19999999, 2.01, 0.57, 0.71, 1.64, 0.28, 2.8...   
2  [0.17, 2.01, 0.57, 0.71, 1.66, 0.26999998, 2.7...   
3  [0.16, 0.57, 1.5899999, 0.71, 1.62, 0.26999